# This model here is trained on 4 selected weather stations to predict one given weather station

## Loading data and converting to dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import datetime
import sklearn

### Solar intensity data

In [2]:
#Load initial data
hourly_sun_intensity = pd.read_excel('2-10_21_524-2 Andmed.xlsx', sheet_name = 'tunni sum.kiirgus', header = 1)

In [3]:
#Update column names by shortening them and converting to English
newColumnNames = dict()
newColumnNames["Aasta"] = "y"
newColumnNames["Kuu"] = "m"
newColumnNames["Päaev"] = "d"
newColumnNames["Kell (UTC)"] = "time"
for columnName in hourly_sun_intensity.columns:
    if "kiirgus" in columnName:
        newColumnNames[columnName] = "solar_"+columnName.replace(" summaarne kiirgus, W/m²", "")
#newColumnNames = ["y", "m", "d", "time"]+["solar_"+columnName.replace(" summaarne kiirgus, W/m²", "") for columnName in hourly_sun_intensity.columns if "kiirgus" in columnName]
hourly_sun_intensity = hourly_sun_intensity.rename(columns=newColumnNames)
#hourly_sun_intensity.columns = newColumnNames

In [4]:
# Some weather stations have changed locations over time, as the differences between their locations are rather small (less than 8 km)
# We at first do not make separation between them

def join_columns(c1, c2, nc, df, column_id): # Function for joining columns, where an area has two weather measuring points
    data = []
    cs = [c1, c2]
    for i, rows in df[cs].iterrows():
        if (pd.isna(rows[0]) == True) & (pd.isna(rows[1]) == False):
            data.append(round(rows[1], 2))
        elif (pd.isna(rows[0]) == False) & (pd.isna(rows[1]) == True):
            data.append(round(rows[0], 2))
        elif (pd.isna(rows[0]) == False) & (pd.isna(rows[1]) == False):
            data.append(round(rows.mean(), 2))
        elif (pd.isna(rows[0]) == True) & (pd.isna(rows[1]) == True):
            data.append(rows[0])

    df = df.drop(columns = [c1, c2])
    df.insert(column_id, nc, data)
    
    return df

In [5]:
#Merge columns, which are due to weather station moving
hourly_sun_intensity = join_columns('solar_Narva', 'solar_Narva-Jõesuu', 'solar_Narva', hourly_sun_intensity, 4)
hourly_sun_intensity = join_columns('solar_Pärnu-Sauga', 'solar_Pärnu', 'solar_Pärnu', hourly_sun_intensity, 5)

In [6]:
#Drop rows where some value is missing
hourly_sun_intensity = hourly_sun_intensity.dropna()
#If value is -1 it corresponds to night, set it to 0
hourly_sun_intensity = hourly_sun_intensity.replace(-1, 0)

In [7]:
#Shift the times -X minutes to facilitate predicting future solar intensity from existing
def shiftDateTime(df, numberOfHours):
    dateTimes = []
    for i in range(len(df)):
        row = df.iloc[i]
        dateTimes+=[datetime.datetime.combine(datetime.date(row.y, row.m, row.d), row.time)+datetime.timedelta(hours=numberOfHours)]
    df2 = copy.deepcopy(df)
    df2["y"] = [date.year for date in dateTimes]
    df2["m"] = [date.month for date in dateTimes]
    df2["d"] = [date.day for date in dateTimes]
    df2["time"] = [date.time() for date in dateTimes]
    
    return df2
    
    

In [8]:
hourly_sun_intensity_Shifted = shiftDateTime(hourly_sun_intensity, -1)

### Data from different weather stations

#### Locations


Tallinn-Harku
Laius: N 59°23´53´´
Pikkus: E 24°36´10´´
Decimal
Lat: 59.398055
Long: 24.602778


Haapsalu meteoroloogiajaam
Laius N 58°56´40´´
Pikkus E 23°33´18´´
Decimal
Lat: 58.944444
Long: 23.555

Narva
Laius: N 59°23´22´´
Pikkus: E 28°06´33´´
Decimal
Lat: 59.389444
Long: 28.109167

Pärnu
Laius: N 58°23´4,44´´
Pikkus: E 24°29´6,71´´
Decimal
Lat: 58.384556
Long: 24.485197

Roomassaare
Laius: N 58°13’05”
Pikkus: E 22°30’23”
Decimal
Lat: 58.218056 
Long: 22.506389 

Tartu-Tõravere meteoroloogiajaam
Laius: N 58°15´51´´
Pikkus: E 26°27´41´
Decimal
Lat: 58.264167
Long: 26.461389

Tiirikoja järvejaam
Laius: N 58°51´55´´
Pikkus: E 26°57´08´´
Decimal
Lat: 58.865278
Long: 26.952222

Vilsandi rannikujaam
Laius: N 58°22´58”
Pikkus: E 21°48´51”
Deciaml
Lat: 58.382778
Long: 21.814167

In [9]:
weather_station_coordinates = dict()
weather_station_coordinates["tallinn"] = [59.398055, 24.602778]
weather_station_coordinates["haapsalu"] = [58.944444, 23.555]
weather_station_coordinates["narva"] = [59.389444, 28.109167]
weather_station_coordinates["parnu"] = [59.389444, 28.109167]
weather_station_coordinates["roomassaare"] = [58.218056, 22.506389]
weather_station_coordinates["tartu"] = [58.264167, 26.461389]
weather_station_coordinates["tiirikoja"] = [58.865278, 26.952222]
weather_station_coordinates["vilsandi"] = [58.382778, 21.814167]

In [10]:
selectedColumns = ["Aasta", "Kuu", "Päev", "Kell (UTC)", "Õhutemperatuur °C","Suhteline õhuniiskus %", "10 minuti keskmine tuule kiirus m/s", "10 minuti keskmine tuule suund"]

In [11]:
#Update column names by shortening them and converting to English

def updateColumnNames(df, location):
    newColumnNames = dict()
    newColumnNames["Aasta"] = "y"
    newColumnNames["Kuu"] = "m"
    newColumnNames["Päev"] = "d"
    newColumnNames["Kell (UTC)"] = "time"
    newColumnNames["Õhutemperatuur °C"] = f"temp_{location}"
    newColumnNames["10 minuti keskmine tuule kiirus m/s"] = f"wind_speed_{location}"
    newColumnNames["Õhurõhk jaama kõrgusel hPa"] = f"pressure_{location}"
    newColumnNames["Suhteline õhuniiskus %"] = f"rel_humidity_{location}"
    newColumnNames["10 minuti keskmine tuule suund"] = f"wind_dir_{location}"
    df = df.rename(columns=newColumnNames)
    return df

In [12]:
def getFromXlsx(filename, columns, location):
    #Load xlsx
    df = pd.read_excel(filename, header = 1)
    df = df[columns]
    #Drop rows where data is missing
    #df = df.dropna()
    #Update column names for clarity
    df = updateColumnNames(df, location)
    return df

In [13]:
#Get Tallinn
data_tallinn = getFromXlsx("./data/Tallinn-Harku_2004-2020.xlsx", selectedColumns, "tallinn")
#data_tallinn["lat_tallinn"] = len(data_tallinn)*[weather_station_coordinates["tallinn"][0]]
#data_tallinn["long_tallinn"] = len(data_tallinn)*[weather_station_coordinates["tallinn"][1]]
#Get Roomassaare
data_roomassaare = getFromXlsx("./data/Roomassaare_2008-2020.xlsx", selectedColumns, "roomassaare")
#data_roomassaare["lat_roomassaare"] = len(data_roomassaare)*[weather_station_coordinates["roomassaare"][0]]
#data_roomassaare["long_roomassaare"] = len(data_roomassaare)*[weather_station_coordinates["roomassaare"][1]]
#Merge tables
data_weather = data_tallinn.merge(data_roomassaare, how='left', on=["y", "m", "d", "time"])

In [14]:
#Get Vilsandi
data_vilsandi = getFromXlsx("./data/Vilsandi_2004-2020.xlsx", selectedColumns, "vilsandi")
#data_vilsandi["lat_vilsandi"] = len(data_vilsandi)*[weather_station_coordinates["vilsandi"][0]]
#data_vilsandi["long_vilsandi"] = len(data_vilsandi)*[weather_station_coordinates["vilsandi"][1]]
#Merge tables
data_weather = data_weather.merge(data_vilsandi, how='left', on=["y", "m", "d", "time"])

In [15]:
##Get Parnu
#data_parnu1 = getFromXlsx("./data/Parnu-Sauga_01.12.2004-31.03.2019.xlsx", selectedColumns, "parnu")
#data_parnu2 = getFromXlsx("./data/Parnu_01.04.2019-2020.xlsx", selectedColumns, "parnu")
#data_parnu = data_parnu1.append(data_parnu2)

##Merge tables
#data_weather = data_weather.merge(data_parnu, how='left', on=["y", "m", "d", "time"])

In [16]:
data_Tartu = getFromXlsx("./data/Tartu-Toravere_2004-2020.xlsx", selectedColumns, "tartu")
#data_Tartu["lat_tartu"] = len(data_Tartu)*[weather_station_coordinates["tartu"][0]]
#data_Tartu["long_tartu"] = len(data_Tartu)*[weather_station_coordinates["tartu"][1]]
#Merge tables

data_weather = data_weather.merge(data_Tartu, how='left', on=["y", "m", "d", "time"])


In [17]:
data_Narva = getFromXlsx("./data/Narva_19.12.2013-2020.xlsx", selectedColumns, "narva")
data_weather = data_weather.merge(data_Narva, how='left', on=["y", "m", "d", "time"])

In [18]:
#Once again drop all rows where some row is missing
data_weather = data_weather.dropna()

### Join weather and solar data

In [19]:
data_solar_weather = hourly_sun_intensity_Shifted.merge(data_weather, how='left', on=["y", "m", "d", "time"])

In [20]:
hours = []
for i in range(len(data_solar_weather)):
    hours+=[data_solar_weather.iloc[i].time.hour]
data_solar_weather["h"] = hours


In [21]:
data_solar_weather = data_solar_weather.dropna()

In [22]:
data_solar_weather.columns

Index(['y', 'm', 'd', 'time', 'solar_Narva', 'solar_Pärnu', 'solar_Haapsalu',
       'solar_Tallinn-Harku', 'solar_Roomassaare', 'solar_Tartu-Tõravere',
       'solar_Tiirikoja', 'solar_Vilsandi', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn',
       'temp_roomassaare', 'rel_humidity_roomassaare',
       'wind_speed_roomassaare', 'wind_dir_roomassaare', 'temp_vilsandi',
       'rel_humidity_vilsandi', 'wind_speed_vilsandi', 'wind_dir_vilsandi',
       'temp_tartu', 'rel_humidity_tartu', 'wind_speed_tartu',
       'wind_dir_tartu', 'temp_narva', 'rel_humidity_narva',
       'wind_speed_narva', 'wind_dir_narva', 'h'],
      dtype='object')

# Let's train models

## 4 other stations, predict Pärnu, different parameters

4 Stations Tartu, Tallinn, Roomassaare, Vilsandi, predict Pärnu
dtr 96.26316991627043
rf 66.69709972042308


### Data Prep

In [75]:
data_solar_weather.columns

Index(['y', 'm', 'd', 'time', 'solar_Narva', 'solar_Pärnu', 'solar_Haapsalu',
       'solar_Tallinn-Harku', 'solar_Roomassaare', 'solar_Tartu-Tõravere',
       'solar_Tiirikoja', 'solar_Vilsandi', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn',
       'temp_roomassaare', 'rel_humidity_roomassaare',
       'wind_speed_roomassaare', 'wind_dir_roomassaare', 'temp_vilsandi',
       'rel_humidity_vilsandi', 'wind_speed_vilsandi', 'wind_dir_vilsandi',
       'temp_tartu', 'rel_humidity_tartu', 'wind_speed_tartu',
       'wind_dir_tartu', 'temp_narva', 'rel_humidity_narva',
       'wind_speed_narva', 'wind_dir_narva', 'h'],
      dtype='object')

In [23]:
X = data_solar_weather[['m','d', 'h', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn',
       'temp_roomassaare', 'rel_humidity_roomassaare',
       'wind_speed_roomassaare', 'wind_dir_roomassaare', 'temp_vilsandi',
       'rel_humidity_vilsandi', 'wind_speed_vilsandi', 'wind_dir_vilsandi',
       'temp_tartu', 'rel_humidity_tartu', 'wind_speed_tartu',
       'wind_dir_tartu']]

y = data_solar_weather[['solar_Pärnu']]

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Random Forest

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [30]:
results_df = pd.DataFrame()

In [31]:
rf= RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


67.20360252881693

In [32]:
############### Random Forest ##############################

n_estimatorss = [100, 150]
max_depths=[20]
min_samples_splits=[4,6,8,16]
seeds = [1]
results_df = pd.DataFrame(columns=['model',"seed","n_estimators","max_depth","min_samples_split",'TrainError', 'ValError', 'deltaErrors'])
for seed in seeds:
    for n_estimator in n_estimatorss:
        for max_d in max_depths:
            for min_ss in min_samples_splits:
                rf = RandomForestRegressor(criterion="squared_error", n_estimators=n_estimator, max_depth=max_d, min_samples_split=min_ss, random_state=seed).fit(X_train, y_train)
                trainError = mean_squared_error(y_train, rf.predict(X_train), squared = False)
                valError = mean_squared_error(y_test, rf.predict(X_test), squared = False)
                results_df = results_df.append({'model': 'RF',"seed":seed,"n_estimators":n_estimator,"max_depth":max_d,"min_samples_split":min_ss,"TrainError":trainError, 'ValError':valError, 'deltaErrors':abs(trainError-valError)}, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
C:\Progr

In [33]:
results_df.sort_values("ValError")

,model,seed,n_estimators,max_depth,min_samples_split,TrainError,ValError,deltaErrors
4,RF,1,150,20,4,30.000551,67.492777,37.492226
0,RF,1,100,20,4,30.161928,67.679470,37.517542
5,RF,1,150,20,6,32.631136,67.693022,35.061885
1,RF,1,100,20,6,32.800755,67.836640,35.035884
6,RF,1,150,20,8,35.261961,67.935102,32.673141
2,RF,1,100,20,8,35.407152,68.095225,32.688073
7,RF,1,150,20,16,44.055066,68.936211,24.881145
3,RF,1,100,20,16,44.152703,69.069536,24.916833


### Gradient Boosting

In [45]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor


############### HistGradientBoostingRegressor ##############################

learning_rates = [0.2, 0.1, 0.05, 0.01]
l2_regularizations=[0, 0.1, 0.2]
results_gb_df = pd.DataFrame(columns=['model',"lr","l2",'TrainError', 'ValError', 'deltaErrors'])
for learning_rate in learning_rates:
    for l2_regularization in l2_regularizations:
        gb = HistGradientBoostingRegressor(l2_regularization=l2_regularization, learning_rate=learning_rate).fit(X_train, y_train.values)
        trainError = mean_squared_error(y_train.values, gb.predict(X_train), squared = False)
        valError = mean_squared_error(y_test.values, gb.predict(X_test), squared = False)
        results_gb_df = results_gb_df.append({'model': 'GB', "lr":learning_rate , "l2":l2_regularization, "TrainError":trainError, 'ValError':valError, 'deltaErrors':abs(trainError-valError)}, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [46]:
results_gb_df.sort_values("ValError")

,model,lr,l2,TrainError,ValError,deltaErrors
1,GB,0.20,0.1,53.368090,67.490322,14.122232
0,GB,0.20,0,53.822786,67.629805,13.807019
2,GB,0.20,0.2,53.803657,67.894717,14.091059
4,GB,0.10,0.1,59.834553,68.489578,8.655025
3,GB,0.10,0,59.789863,68.525111,8.735248
5,GB,0.10,0.2,59.649989,68.600153,8.950164
8,GB,0.05,0.2,66.175811,71.010685,4.834874
7,GB,0.05,0.1,66.154377,71.108425,4.954048
6,GB,0.05,0,66.104724,71.113731,5.009007
9,GB,0.01,0,108.877045,108.870004,0.007041


## SVR

In [48]:
from sklearn import svm

clf = svm.SVR()
clf.fit(X_train, y_train.values)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [49]:
mean_squared_error(y_test.values, clf.predict(X_test), squared = False)

188.0323328276874

## Lasso

In [52]:
from sklearn import linear_model

In [57]:
lasso = linear_model.Lasso(alpha=0.01)
lasso.fit(X_train, y_train.values)

Lasso(alpha=0.01)

In [58]:
mean_squared_error(y_test.values, lasso.predict(X_test), squared = False)

141.81088469396633

## Ridge

In [59]:
ridge = linear_model.Ridge(alpha=0.01)
ridge.fit(X_train, y_train.values)

Ridge(alpha=0.01)

In [60]:
mean_squared_error(y_test.values, ridge.predict(X_test), squared = False)

141.81099217260274

## Predict Pärnu, use 4 stations, use only one type of variables

## Temperature

In [115]:
X = data_solar_weather[['m','d', 'h', 'temp_tallinn',
       'temp_roomassaare', 'temp_vilsandi',
       'temp_tartu']]

y = data_solar_weather[['solar_Pärnu']]

In [116]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [117]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [118]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth = 80)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=80)

In [119]:
dtr.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dtr.predict(X_test_scaled), squared = False)

107.92307200732515

In [120]:
results_df = pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


77.96527929643392

In [121]:
rf= RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


77.86844172490507

## Humidity

In [123]:
X = data_solar_weather[['m','d', 'h',
       'rel_humidity_tallinn', 'rel_humidity_roomassaare',
       'rel_humidity_vilsandi', 'rel_humidity_tartu']]

y = data_solar_weather[['solar_Pärnu']]

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [125]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [126]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth = 80)
dtr.fit(X_train, y_train)
dtr.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dtr.predict(X_test_scaled), squared = False)

103.89287296065847

In [127]:
results_df = pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


72.07701172527304

### Idea of trying Stacking to see if it works better

In [128]:
from sklearn.ensemble import StackingRegressor

In [ ]:
estimators = [
    ('lr', RidgeCV()),
    ('svr', LinearSVR(random_state=42))
]